<a href="https://colab.research.google.com/github/DayByDayBy/whypy/blob/main/why_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   whyPy
---
### a Jupyter Notebook and LLM botherer



playing with iterative invocation of large language models and collecting the responses

(altho right now it's just a skeleton, prep)


In [ ]:
#@title installations

# installations required to run the notebook

!pip install argparse pandas sentence_transformers langchain_community langchain langchain_core datetime matplotlib

In [ ]:
#@title imports

# imports to run first (some common, some less so, may be possible to skip this step depending

import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from datetime import datetime
import time

In [ ]:
#@title ipy input

from ipywidgets import IntText, interact

iterations = IntText(value=10, description='Iterations:')
interact(lambda iterations: iterations, iterations=iterations)

In [ ]:
#@title main function

%%time